In [1]:
import sys
import os
from pprint import pprint

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from resume_scanner.parsing.parsing import parse_resume
from resume_scanner.scoring.scoring import score_resume

In [33]:
parsed_resume = parse_resume("../data/input/resumes/Kareem_resume.pdf")

In [30]:
with open("../data/input/jobs/c1_swe.txt", "r") as file:
    job_desc = file.read()

In [34]:
scored_resume = score_resume(parsed_resume, job_desc)

In [35]:
pprint(scored_resume.model_dump())

{'final_score': 0.74,
 'resume_eval': {'education': {'alignment': 5,
                               'comment': 'The candidate is pursuing a '
                                          'Bachelor of Science in Computer '
                                          'Science with a strong GPA and '
                                          'relevant coursework, meeting the '
                                          'educational requirements for the '
                                          'internship.'},
                 'experience': {'comment': 'The candidate has relevant '
                                           'experience in machine learning and '
                                           'software development, which aligns '
                                           "well with the internship's focus "
                                           'on technology and programming. The '
                                           'contributions demonstrate '
                         